In [7]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import numpy as np

import os
import scipy.io


In [10]:
# Path to images and annotations
path_images = "101_ObjectCategories/airplanes/"
path_annot = "Annotations/Airplanes_Side_2/"


# list of paths to images and annotations
image_paths = [
    f for f in os.listdir(path_images) if os.path.isfile(os.path.join(path_images, f))
]
annot_paths = [
    f for f in os.listdir(path_annot) if os.path.isfile(os.path.join(path_annot, f))
]

image_paths.sort()
annot_paths.sort()

image_size = 224  # resize input images to this size

images, targets = [], []

# loop over the annotations and images, preprocess them and store in lists
for i in range(0, len(annot_paths)):
    # Access bounding box coordinates
    annot = scipy.io.loadmat(path_annot + annot_paths[i])["box_coord"][0]

    top_left_x, top_left_y = annot[2], annot[0]
    bottom_right_x, bottom_right_y = annot[3], annot[1]

    image = keras.utils.load_img(
        path_images + image_paths[i],
    )
    (w, h) = image.size[:2]

    # resize train set images
    # if i < int(len(annot_paths) * 0.8):
        # resize image if it is for training dataset
    image = image.resize((image_size, image_size))

    # convert image to array and append to list
    images.append(keras.utils.img_to_array(image))

    # apply relative scaling to bounding boxes as per given image and append to list
    targets.append(
        (
            float(top_left_x) / w,
            float(top_left_y) / h,
            float(bottom_right_x) / w,
            float(bottom_right_y) / h,
        )
    )



In [11]:
(x_train), (y_train) = (
    np.asarray(images[: int(len(images) * 0.8)]),
    np.asarray(targets[: int(len(targets) * 0.8)]),
)

(x_test), (y_test) = (
    np.asarray(images[int(len(images) * 0.8) :]),
    np.asarray(targets[int(len(targets) * 0.8) :]),
)

In [12]:
def conv_block(x, filters=16, kernel_size=3, strides=2):
    conv_layer = layers.Conv2D(filters, kernel_size, strides=strides, activation=tf.nn.swish, use_bias=False, padding="same", groups=1)

    return conv_layer(x)


def inverted_residual_block(x, expanded_channels, output_channels, strides=1):
    m = layers.Conv2D(expanded_channels, 1, padding="same", use_bias=False, groups=1)(x)
    m = layers.BatchNormalization()(m)

    m = layers.DepthwiseConv2D(3, strides=strides, padding="same", use_bias=False)(m)
    m = layers.BatchNormalization()(m)
    m = tf.nn.swish(m)

    m = layers.Conv2D(output_channels, 1, padding="same", use_bias=False, groups=1)(m)
    m = layers.BatchNormalization()(m)

    if strides == 1:
        return layers.Concatenate(axis=-1)([m, x])
    elif strides == 2:
        x = layers.Conv2D(output_channels, kernel_size=(2, 2), strides=2, padding="same", groups=1)(x)
        return layers.Concatenate(axis=-1)([m, x])

    return m


def ffn(x, hidden_units, dropout_rate, use_bias=False):
    a = tf.reshape(x, (-1, 1, x.shape[1], x.shape[-1]))

    for hidden_unit in hidden_units:
        a = tf.keras.layers.Conv2D(filters=hidden_unit, kernel_size=1, activation=tf.nn.swish, padding='same', use_bias=use_bias, groups=1)(a)
        a = layers.Dropout(dropout_rate)(a)

    x = tf.reshape(a, (-1, x.shape[1], x.shape[-1]))

    return x


def transformer_block(x, transformer_layers, projection_dim, num_heads=2):
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim, dropout=0.0, use_bias=False)(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, x])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        # x3 = mlp(x3, hidden_units=[x.shape[-1] * 2, x.shape[-1]], dropout_rate=0.1,)
        x3 = ffn(x3, hidden_units=[x.shape[-1] * 2, x.shape[-1]], dropout_rate=0.0,)
        # Skip connection 2.
        x = layers.Add()([x3, x2])

    return x


def mobilevit_block(x, num_blocks, projection_dim, strides=1):
    # Local projection with convolutions.
    local_features = conv_block(x, filters=projection_dim, strides=strides)
    local_features = conv_block(local_features, filters=projection_dim, kernel_size=1, strides=strides)

    non_overlapping_patches = tf.reshape(local_features, (-1, tf.shape(local_features)[1] * tf.shape(local_features)[2], tf.shape(local_features)[-1]))

    global_features = transformer_block(non_overlapping_patches, num_blocks, projection_dim)

    folded_feature_map = tf.reshape(global_features, (-1, tf.shape(local_features)[1], tf.shape(local_features)[2], tf.shape(local_features)[-1]))


    # Apply point-wise conv -> concatenate with the input features.
    folded_feature_map = conv_block(folded_feature_map, filters=x.shape[-1], kernel_size=1, strides=strides)
    local_global_features = layers.Concatenate(axis=-1)([x, folded_feature_map])

    # Fuse the local and global features using a convolution layer.
    local_global_features = conv_block(local_global_features, filters=projection_dim, strides=strides)

    return local_global_features


In [13]:
def create_mobilevit(num_classes=4, expansion_factor=2, image_size=224):

    inputs = keras.Input((image_size, image_size, 3))
    x = layers.Rescaling(scale=1.0 / 255)(inputs)

    # Initial conv-stem -> MV2 block.
    x = layers.Conv2D(filters=16, kernel_size=16, strides=16, activation=tf.nn.swish, use_bias=False, padding="same")(x)
    x = inverted_residual_block(x, expanded_channels=16 * expansion_factor, output_channels=16)

    # Downsampling with MV2 block.
    x = inverted_residual_block(x, expanded_channels=16 * expansion_factor, output_channels=24, strides=2)
    x = inverted_residual_block(x, expanded_channels=24 * expansion_factor, output_channels=24)
    x = inverted_residual_block(x, expanded_channels=24 * expansion_factor, output_channels=24)

    # First MV2 -> MobileViT block.
    x = inverted_residual_block(x, expanded_channels=24 * expansion_factor, output_channels=48, strides=2)
    x = mobilevit_block(x, num_blocks=2, projection_dim=64)

    # Second MV2 -> MobileViT block.
    x = inverted_residual_block(x, expanded_channels=64 * expansion_factor, output_channels=64, strides=2)
    x = mobilevit_block(x, num_blocks=4, projection_dim=80)

    # Third MV2 -> MobileViT block.
    x = inverted_residual_block(x, expanded_channels=80 * expansion_factor, output_channels=80, strides=2)
    x = mobilevit_block(x, num_blocks=3, projection_dim=96)
    x = conv_block(x, filters=320, kernel_size=1, strides=1)
    # Classification head.
    x = layers.GlobalAvgPool2D()(x)
    bounding_box = layers.Dense(num_classes)(x)

    model = keras.Model(inputs, bounding_box)

    return model

In [14]:
class MeanIoU(tf.keras.metrics.Metric):
    def __init__(self, name="mean_iou", **kwargs):
        super(MeanIoU, self).__init__(name=name, **kwargs)
        self.intersection = self.add_weight(name="intersection", initializer="zeros")
        self.union = self.add_weight(name="union", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        # Get the intersection coordinates of the bounding boxes
        top_left = tf.maximum(y_true[:, :2], y_pred[:, :2])
        bottom_right = tf.minimum(y_true[:, 2:], y_pred[:, 2:])

        # Compute the intersection area
        intersection_dims = tf.maximum(bottom_right - top_left, 0)
        intersection_area = intersection_dims[:, 0] * intersection_dims[:, 1]

        # Compute the area of the ground truth and predicted bounding boxes
        y_true_area = (y_true[:, 2] - y_true[:, 0]) * (y_true[:, 3] - y_true[:, 1])
        y_pred_area = (y_pred[:, 2] - y_pred[:, 0]) * (y_pred[:, 3] - y_pred[:, 1])

        # Compute the union area
        union_area = y_true_area + y_pred_area - intersection_area

        # Update the intersection and union sums
        self.intersection.assign_add(tf.reduce_sum(intersection_area))
        self.union.assign_add(tf.reduce_sum(union_area))

    def result(self):
        return self.intersection / (self.union + tf.keras.backend.epsilon())

    def reset_state(self):
        self.intersection.assign(0)
        self.union.assign(0)

In [16]:
def run_experiment(model, learning_rate, weight_decay, batch_size, num_epochs):

    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    # Compile model.
    model.compile(optimizer=optimizer, loss=keras.losses.MeanSquaredError(), metrics=[MeanIoU()])

    checkpoint_filepath = "checkpoints"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_mean_iou",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[
            checkpoint_callback,
            keras.callbacks.EarlyStopping(monitor="val_loss", patience=20),
        ],
    )

    return history


input_shape = (image_size, image_size, 3)  # input image shape
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 100


vit_object_detector = create_mobilevit(num_classes=4, expansion_factor=2)
# Train model
history = run_experiment(
    vit_object_detector, learning_rate, weight_decay, batch_size, num_epochs
)

Epoch 1/100


2023-04-23 21:26:14.863850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:417] Loaded runtime CuDNN library: 8.1.1 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2023-04-23 21:26:14.864474: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops.cc:1068 : UNIMPLEMENTED: DNN library is not found.
2023-04-23 21:26:14.864492: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): UNIMPLEMENTED: DNN library is not found.
	 [[{{node model_1/conv2d_50/Conv2D}}]]


UnimplementedError: Graph execution error:

Detected at node 'model_1/conv2d_50/Conv2D' defined at (most recent call last):
    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/h